In [1]:
from utils import get_root_dir

/home/erlendlokna/Documents/ntnu/Generative-SSL-VQVAE-modelling/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
d = get_root_dir()

In [30]:
SSL_method = 'vicreg-1'
dataset_name = 'CBF'
t = get_root_dir().joinpath('saved_models'), f'{SSL_method}_encoder-CBF.ckpt'

In [31]:
import os
os.path.exists(t)

TypeError: stat: path should be string, bytes, os.PathLike or integer, not tuple

In [29]:
os.path.exists()

<bound method Path.exists of PosixPath('/home/erlendlokna/Documents/ntnu/Generative-SSL-VQVAE-modelling/saved_models/vicreg-1_encoder-CBF.ckpt')>